with filtering:
driver = 5
seed = 1337
epoch = 50
adam lr = 0.00005
worst rate: 0.7188 (epoch 2)
best rate: 0.9688 (epoch 25)
loss: 0.2239 (epoch: 27) -- 0.22391266226768494


((tensor(0.7188), 2), (tensor(0.9688), 25), (0.22391266226768494, 27))

In [1]:
import os
import torch
import pickle

import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

from ds.drivers.datasets import DriverDataset, DriversDataset, all_data

In [2]:
seed = 1337
np.random.seed(seed)
torch.random.manual_seed(seed)

In [3]:
base_path = "/home/dmo/Documents/human_func_state"
models_dumps = os.path.join(base_path, "models_dumps")

In [4]:
epoch_count = 200
adam_lr = 0.0001
train_batch_size = 8
test_batch_size = 1
ds_step_size = 5
device = "cuda"

In [5]:
class ConvX(nn.Module):
    def __init__(
        self, in_planes, out_planes, kernel=3, stride=1, padding=None
    ):
        super(ConvX, self).__init__()
        padding = kernel // 2 if padding is None else padding
        self.conv = nn.Conv1d(
            in_planes,
            out_planes,
            kernel_size=kernel,
            stride=stride,
            padding=padding,
            bias=False,
        )
        self.bn = nn.BatchNorm1d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [6]:
class Net6Conv2MaxPool(nn.Module):
    def __init__(self):
        super(Net6Conv2MaxPool, self).__init__()
        self.seq = torch.nn.Sequential(
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            nn.MaxPool1d(2),
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            nn.MaxPool1d(2),
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, stride=2),
            nn.Conv1d(1, 1, kernel_size=2),
        )
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        return self.softmax(torch.flatten(self.seq(x), 1))

In [7]:
with open(
    os.path.join(
        models_dumps, "Net6Conv2MaxPool_ASGD_lr_0.0001_betas_default.pkl"
    ),
    "rb",
) as f:
    data = pickle.load(f)
data.get("rate")

tensor(0.8159)

In [8]:
def get_net_and_optimizer():
    net = Net6Conv2MaxPool()
    net.load_state_dict(data.get("net_state_dict"))
    children = list(net.seq.children())
    for i, child in enumerate(children):
        if i != len(children) - 1:
            for param in child.parameters():
                param.requires_grad = False
    net = net.to(device=device)
    optimizer = optim.Adam(net.seq[-1].parameters(), lr=adam_lr)
    return net, optimizer

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
def noop(*_args, **_kwargs):
    pass

In [11]:
def val(model, dl_test, return_valid_count_only=True):
    labels_ = []
    outputs_ = []
    with torch.no_grad():
        sum_ = 0
        for i, data in enumerate(dl_test):
            inputs, labels = data
            labels_.append(labels)
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )
            outputs = model(inputs).cpu()
            outputs = outputs.max(1).indices
            outputs_.append(outputs)
            eq = outputs == labels
            sum_ += eq.sum()

    return sum_ if return_valid_count_only else (sum_, labels_, outputs_)

In [12]:
def train(
    model,
    optimizer,
    dl_train,
    dl_test,
    epoch_count=50,
    print_step=50,
    print_f=noop,
):
    min_loss = (np.inf, 0)
    best_rate = (0, 0)
    worst_rate = (1.0, 0)
    len_test = len(dl_test) * dl_test.batch_size
    for epoch in range(epoch_count):  # loop over the dataset multiple times
        running_loss = 0.0
        model.train()
        for i, data in enumerate(dl_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.cpu(), labels)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        mean_loss = running_loss / print_step
        if mean_loss < min_loss[0]:
            min_loss = (mean_loss, epoch)
        count_of_correct = val(model, dl_test)
        rate = count_of_correct / len_test
        if rate > best_rate[0]:
            best_rate = (rate, epoch)
        if rate < worst_rate[0]:
            worst_rate = (rate, epoch)

        print_f(
            f"[{epoch + 1}] "
            f"rate: {rate:.4f} - {count_of_correct:3d}/{len_test}; "
            f"{best_rate = }, {worst_rate = }, loss: {mean_loss:.3f}"
        )
    return worst_rate, best_rate, min_loss

In [13]:
ds_all_test = DriversDataset(
    all_data,
    ds_type="test",
    data_field="working_data_filtered",
    step=ds_step_size,
)
dl_all_test = DataLoader(
    ds_all_test,
    batch_size=1,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [14]:
net, optimizer = get_net_and_optimizer()
net.eval()
print("All")
acc = val(net, dl_all_test)
print("Rate: ", acc / len(ds_all_test))

All
Rate:  tensor(0.8159)


In [15]:
drivers_test_datasets = {
    driver_id: DriverDataset(
        driver_data,
        ds_type="test",
        data_field="working_data_filtered",
        step=ds_step_size,
    )
    for driver_id, driver_data in all_data.items()
}

In [16]:
accs = []
for driver_id, ds_test in drivers_test_datasets.items():
    # ds_train = DriverDataset(driver_data, ds_type="train", data_field="working_data_filtered")
    # ds_test = DriverDataset(driver_data, ds_type="test", data_field="working_data_filtered", step=ds_step_size)

    # dl_train = DataLoader(
    #     ds_train,
    #     batch_size = train_batch_size,
    #     shuffle = True,
    #     num_workers = 1,
    #     pin_memory = False,
    #     drop_last = True
    # )
    dl_test = DataLoader(
        ds_test,
        batch_size=test_batch_size,
        shuffle=True,
        num_workers=1,
        pin_memory=False,
        drop_last=True,
    )
    print("-------------------------")
    net, optimizer = get_net_and_optimizer()
    net.eval()
    print("Driver", driver_id)
    acc = val(net, dl_test)
    accs.append(acc / len(ds_test))
    print("rate before: ", accs[-1])
    # worst, best, loss = train(net, optimizer, dl_train, dl_test, epoch_count=epoch_count)
    # print(f"{best = }, {worst = }, {loss = }")

-------------------------
Driver 5
rate before:  tensor(0.8155)
-------------------------
Driver 6
rate before:  tensor(0.)
-------------------------
Driver 7
rate before:  tensor(0.3175)
-------------------------
Driver 8
rate before:  tensor(1.)
-------------------------
Driver 9
rate before:  tensor(0.9756)
-------------------------
Driver 10
rate before:  tensor(1.)
-------------------------
Driver 11
rate before:  tensor(0.9588)
-------------------------
Driver 12
rate before:  tensor(0.9296)
-------------------------
Driver 15
rate before:  tensor(0.9189)
-------------------------
Driver 16
rate before:  tensor(1.)


In [17]:
accs

[tensor(0.8155),
 tensor(0.),
 tensor(0.3175),
 tensor(1.),
 tensor(0.9756),
 tensor(1.),
 tensor(0.9588),
 tensor(0.9296),
 tensor(0.9189),
 tensor(1.)]

In [18]:
# driver_id = 6
# driver_data = all_data.get(driver_id)
# ds_train = DriverDataset(driver_data, ds_type="train", data_field="working_data_filtered")
# ds_test = DriverDataset(driver_data, ds_type="test", data_field="working_data_filtered")
#
# dl_train = DataLoader(
#     ds_train,
#     batch_size = train_batch_size,
#     shuffle = True,
#     num_workers = 1,
#     pin_memory = False,
#     drop_last = True
# )
# dl_test = DataLoader(
#     ds_test,
#     batch_size = test_batch_size,
#     shuffle = True,
#     num_workers = 1,
#     pin_memory = False,
#     drop_last = True
# )
# print("-------------------------")
# net, optimizer = get_net_and_optimizer()
# print("Driver", driver_id)
# print("rate before: ", val(net, dl_test) / len(ds_test))
# worst, best, loss = train(net, optimizer, dl_train, dl_test, epoch_count=epoch_count)
# print(f"{best = }, {worst = }, {loss = }")

In [19]:
sum(accs)

tensor(7.9159)

In [20]:
print("\n".join(map(str, zip(drivers_test_datasets.keys(), accs))))

(5, tensor(0.8155))
(6, tensor(0.))
(7, tensor(0.3175))
(8, tensor(1.))
(9, tensor(0.9756))
(10, tensor(1.))
(11, tensor(0.9588))
(12, tensor(0.9296))
(15, tensor(0.9189))
(16, tensor(1.))


In [21]:
ds_all_test.__len__()

945

In [22]:
print(
    "\n".join(
        map(
            str,
            zip(
                drivers_test_datasets.keys(),
                map(len, drivers_test_datasets.values()),
            ),
        )
    )
)

(5, 103)
(6, 92)
(7, 63)
(8, 132)
(9, 82)
(10, 137)
(11, 97)
(12, 71)
(15, 111)
(16, 57)
